ERROR: You must give at least one requirement to install (see "pip help install")


In [1]:
import os
import numpy as np
import torch
import pandas as pd
from functools import reduce
from transformers import AutoTokenizer, RobertaTokenizer

In [2]:
device = torch.device("cpu")

In [3]:
d_path = os.path.join(os.getcwd(), "..", "..", "token-classes-dataset")

In [4]:
# list of partial dataframes
dfs = []

# collects partial dataframes from oracles
for file_name in os.listdir(d_path):
    df = pd.read_json(os.path.join(d_path,  file_name))
    dfs.append(df)

df_dataset = pd.concat(dfs)
# drop column id (it is not relevant for training the model)
df_dataset = df_dataset.drop(['id'], axis=1)
# map empty cells to empty strings
df_dataset.fillna('', inplace=True)
# specify the type of each column in the dataset
df_dataset = df_dataset.astype({
    'label': 'bool',
    'oracleId': 'int64',
    'oracleType': 'string',
    'projectName': 'string',
    'packageName': 'string',
    'className': 'string',
    'javadocTag': 'string',
    'methodJavadoc': 'string',
    'methodSourceCode': 'string',
    'classJavadoc': 'string',
    'classSourceCode': 'string',
    'oracleSoFar': 'string',
    'token': 'string',
    'tokenClass': 'string',
    'tokenInfo': 'string'
})

In [5]:
df_dataset.shape

(189187, 15)

In [6]:
df_dataset.head()

,label,oracleId,oracleType,projectName,packageName,className,javadocTag,methodJavadoc,methodSourceCode,classJavadoc,classSourceCode,oracleSoFar,token,tokenClass,tokenInfo
0,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,MethodArgument,
1,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,Class,
2,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,This,
3,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,TRUE,
4,True,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,Semicolon,


In [7]:
unique_values = df_dataset['tokenClass'].unique()

In [8]:
unique_values

<StringArray>
[        'MethodArgument',                  'Class',                   'This',
                   'TRUE',              'Semicolon',     'OpeningParenthesis',
            'ArraysClass',           'IneqOperator',             'EqOperator',
                 'Period',     'InstanceOfOperator',                   'NULL',
        'LogicalOperator',      'NonEqIneqOperator',                  'S_INT',
  'BitwiseNegateOperator',           'QuestionMark',         'MethodResultID',
             'MethodName',     'ClosingParenthesis',                  'Colon',
                  'FALSE',           'StreamMethod',                'BOOLEAN',
                  'Comma',             'ClassField', 'BitwiseLogicalOperator',
   'ArithmeticalOperator',   'BitwiseShiftOperator',                 'DOUBLE',
            'MatchMethod',         'MatchMethodVar',             'RightArrow',
          'ClassModifier',               'S_STRING']
Length: 35, dtype: string

In [9]:
df_dataset.columns

Index(['label', 'oracleId', 'oracleType', 'projectName', 'packageName',
       'className', 'javadocTag', 'methodJavadoc', 'methodSourceCode',
       'classJavadoc', 'classSourceCode', 'oracleSoFar', 'token', 'tokenClass',
       'tokenInfo'],
      dtype='object')

In [10]:
# delete the oracle ids and the tgt labels from the input dataset
df_src = df_dataset.drop(['label','oracleId','projectName','classSourceCode','classJavadoc','token','tokenInfo'], axis=1)

In [11]:
df_src.shape

(189187, 8)

In [12]:
%%capture
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=False) #AutoTokenizer.from_pretrained("microsoft/codebert-base", model_max_length=512)

In [23]:
df_src_concat = df_src.apply(lambda row: tokenizer.tokenize(tokenizer.cls_token.join(row.values)), axis=1)
# The pandas dataframe is transformed in a list of strings: each string is a input
# to the model
src = df_src_concat.to_numpy().tolist()

#max_len = reduce(lambda max_len, s: len(s) if len(s) > max_len else max_len, src,0) + 2

In [28]:
%%capture
tokenizer_2 = AutoTokenizer.from_pretrained("microsoft/codebert-base", model_max_length=512)


In [30]:
df_src_concat_2 = df_src.apply(lambda row: tokenizer_2.tokenize(tokenizer.cls_token.join(row.values)), axis=1)
# The pandas dataframe is transformed in a list of strings: each string is a input
# to the model
src_2 = df_src_concat.to_numpy().tolist()

#max_len = reduce(lambda max_len, s: len(s) if len(s) > max_len else max_len, src,0) + 2

Token indices sequence length is longer than the specified maximum sequence length for this model (653 > 512). Running this sequence through the model will result in indexing errors


In [14]:
unique_values = df_dataset['tokenClass'].unique()

In [15]:
unique_values

<StringArray>
[        'MethodArgument',                  'Class',                   'This',
                   'TRUE',              'Semicolon',     'OpeningParenthesis',
            'ArraysClass',           'IneqOperator',             'EqOperator',
                 'Period',     'InstanceOfOperator',                   'NULL',
        'LogicalOperator',      'NonEqIneqOperator',                  'S_INT',
  'BitwiseNegateOperator',           'QuestionMark',         'MethodResultID',
             'MethodName',     'ClosingParenthesis',                  'Colon',
                  'FALSE',           'StreamMethod',                'BOOLEAN',
                  'Comma',             'ClassField', 'BitwiseLogicalOperator',
   'ArithmeticalOperator',   'BitwiseShiftOperator',                 'DOUBLE',
            'MatchMethod',         'MatchMethodVar',             'RightArrow',
          'ClassModifier',               'S_STRING']
Length: 35, dtype: string

In [17]:
df_src.head()

,oracleType,packageName,className,javadocTag,methodJavadoc,methodSourceCode,oracleSoFar,tokenClass
0,NORMAL_POST,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,,MethodArgument
1,NORMAL_POST,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,,Class
2,NORMAL_POST,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,,This
3,NORMAL_POST,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,,TRUE
4,NORMAL_POST,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,,Semicolon


In [31]:
max(src)

['PRE',
 '<s>',
 'pl',
 'ume',
 '<s>',
 'Weak',
 'Id',
 'entity',
 'P',
 'air',
 '<s>',
 '@',
 'param',
 'Ġb',
 'Ġsecond',
 'Ġargument',
 '<s>',
 'ĉ',
 '/**',
 'ĠFactory',
 'Ġmethod',
 'Ġwith',
 'Ġshort',
 'Ġname',
 'Ġand',
 'Ġno',
 'Ġneed',
 'Ġto',
 'Ġname',
 'Ġtype',
 'Ġparameters',
 '.',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ*',
 'Ġ@',
 'param',
 'Ġ<',
 'A',
 '>',
 'Ġtype',
 'Ġof',
 'Ġfirst',
 'Ġargument',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ*',
 'Ġ@',
 'param',
 'Ġ<',
 'B',
 '>',
 'Ġtype',
 'Ġof',
 'Ġsecond',
 'Ġargument',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ*',
 'Ġ@',
 'param',
 'Ġa',
 'Ġfirst',
 'Ġargument',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ*',
 'Ġ@',
 'param',
 'Ġb',
 'Ġsecond',
 'Ġargument',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ*',
 'Ġ@',
 'return',
 'Ġa',
 'ĠWeak',
 'Id',
 'entity',
 'P',
 'air',
 'Ġof',
 'Ġ(',
 'a',
 ',',
 'Ġb',
 ')',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ*/',
 '<s>',
 'public',
 'Ġstatic',
 'Ġ<',
 'A',
 'Ġextends',
 'ĠObject',
 ',',
 'ĠB',
 'Ġextends',
 'ĠObject',
 '>',
 'ĠWeak',
 'Id',
 'entity',
 'P',
 'air',
 '<',
 'A',
 ',',
 'ĠB',
 '>',
 'Ġof'

In [32]:
# Assuming you have a list of strings named 'string_list'
max_length_element = max(src_2, key=lambda x: len(x))

# If you want to get the index of the element instead, you can use argmax()
max_length_index = np.argmax([len(x) for x in src_2])

In [33]:
max_length_element

['NOR',
 'MAL',
 '_',
 'POST',
 '<s>',
 'org',
 '.',
 'apache',
 '.',
 'comm',
 'ons',
 '.',
 'math',
 '3',
 '.',
 'optim',
 '.',
 'non',
 'linear',
 '.',
 'sc',
 'al',
 'ar',
 '.',
 'n',
 'oder',
 'iv',
 '<s>',
 'BO',
 'BY',
 'Q',
 'A',
 'Opt',
 'im',
 'izer',
 '<s>',
 'Ġthe',
 'Ġvalue',
 'Ġof',
 'Ġthe',
 'Ġobjective',
 'Ġat',
 'Ġthe',
 'Ġoptimum',
 '.',
 '<s>',
 'ĉ',
 '/**',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ*',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'ĠThe',
 'Ġarguments',
 'ĠN',
 ',',
 'ĠN',
 'PT',
 ',',
 'ĠX',
 ',',
 'ĠXL',
 ',',
 'ĠX',
 'U',
 ',',
 'ĠRH',
 'OB',
 'EG',
 ',',
 'ĠR',
 'HO',
 'END',
 ',',
 'ĠI',
 'PR',
 'INT',
 'Ġand',
 'ĠMAX',
 'FUN',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ*',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġare',
 'Ġidentical',
 'Ġto',
 'Ġthe',
 'Ġcorresponding',
 'Ġarguments',
 'Ġin',
 'ĠSUB',
 'R',
 'OUT',
 'INE',
 'ĠB',
 'OB',
 'Y',
 'Q',
 'A',
 '.',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ*',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'ĠX',
 'B',
 'ASE',
 'Ġholds',
 'Ġa',
 'Ġshift',
 'Ġof',
 'Ġorigi

In [34]:
len(max_length_element)

28619

In [22]:
b = []

b.append(max_length_element)

a = tokenizer(
    b,
    truncation=False
)

ValueError: too many values to unpack (expected 2)

In [ ]:
len(a["input_ids"][0])

268

In [ ]:
# Assuming you have a string named 'my_string'
numerical_representation = [ord(char) for char in my_string]  # Convert string to a list of ASCII values

# Create a PyTorch tensor from the numerical representation
tensor = torch.tensor(numerical_representation)

TypeError: new(): invalid data type 'str'

In [ ]:
type(tokenizer.tokenize(max_length_element)[0])

str

In [ ]:
max_len

268

In [35]:
string_lengths = np.array([len(sublist) + 2 for sublist in src ])

max_length = np.max(string_lengths)
mean_length = np.mean(string_lengths)
median_length = np.median(string_lengths)

In [36]:
max_length

28621

In [37]:
mean_length

391.1331011115986

In [38]:
median_length

292.0

In [39]:
filtered_strings = [string for string in src if len(string) < 512]

In [40]:
len(src)

189187

In [41]:
len(filtered_strings)

155578

In [42]:
len(df_src["tokenClass"].value_counts())

35

In [19]:
df_dataset["tokenClass"].value_counts()#/len(df_dataset["tokenClass"]) * 100

Class                     32428
Semicolon                 31038
MethodArgument            29226
OpeningParenthesis        29003
This                      14980
TRUE                      12001
ArraysClass                6109
LogicalOperator            5729
EqOperator                 5149
IneqOperator               4098
Period                     3778
InstanceOfOperator         2651
NULL                       2358
S_INT                      2212
ClosingParenthesis         1368
NonEqIneqOperator          1184
MethodResultID              961
DOUBLE                      887
BitwiseNegateOperator       841
FALSE                       828
MethodName                  523
Colon                       386
QuestionMark                296
ArithmeticalOperator        253
ClassField                  239
BitwiseLogicalOperator      186
BitwiseShiftOperator        186
BOOLEAN                     138
StreamMethod                 52
MatchMethodVar               42
MatchMethod                  14
RightArr

In [17]:
np.bincount(df_src["tokenClass"])

ValueError: invalid literal for int() with base 10: 'MethodArgument'

In [14]:
# Get unique values from the column
unique_values = df_dataset['tokenClass'].unique()

# Create a dictionary to map string values to their corresponding vector
mapping = {}
for i, value in enumerate(unique_values):
    vector = np.zeros(len(unique_values))
    vector[i] = 1.0
    mapping[value] = list(vector)

# Add a new column to the original DataFrame with the mapped vectors
df_dataset['tokenClassVectorized'] = df_dataset['tokenClass'].map(mapping)

In [15]:
df_dataset.head()

,label,oracleId,oracleType,projectName,packageName,className,javadocTag,methodJavadoc,methodSourceCode,classJavadoc,classSourceCode,oracleSoFar,token,tokenClass,tokenInfo,tokenClassVectorized
0,False,15264,PRE,plume-lib-1.1.0,plume,Lookup,@param reader where to read the entry from,/** * Returns the next entry. If no more ...,public static Entry old_get_entry(EntryReader ...,"/** * Lookup searches a set of files, much li...",package plume; import static plume.EntryReade...,,,MethodArgument,,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,False,15264,PRE,plume-lib-1.1.0,plume,Lookup,@param reader where to read the entry from,/** * Returns the next entry. If no more ...,public static Entry old_get_entry(EntryReader ...,"/** * Lookup searches a set of files, much li...",package plume; import static plume.EntryReade...,,,Class,,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,True,15264,PRE,plume-lib-1.1.0,plume,Lookup,@param reader where to read the entry from,/** * Returns the next entry. If no more ...,public static Entry old_get_entry(EntryReader ...,"/** * Lookup searches a set of files, much li...",package plume; import static plume.EntryReade...,,,Semicolon,,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,False,15264,PRE,plume-lib-1.1.0,plume,Lookup,@param reader where to read the entry from,/** * Returns the next entry. If no more ...,public static Entry old_get_entry(EntryReader ...,"/** * Lookup searches a set of files, much li...",package plume; import static plume.EntryReade...,,,OpeningParenthesis,,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,False,15265,NORMAL_POST,plume-lib-1.1.0,plume,Lookup,"the next entry, or null",/** * Returns the next entry. If no more ...,public static Entry old_get_entry(EntryReader ...,"/** * Lookup searches a set of files, much li...",package plume; import static plume.EntryReade...,,,MethodArgument,,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [18]:
# Convert string labels to integer labels
unique_labels, integer_labels = np.unique(df_dataset["tokenClass"], return_inverse=True)

# Count the occurrences of each class
class_counts = np.bincount(integer_labels)

# Calculate the inverse class frequencies
total_samples = np.sum(class_counts)
class_frequencies = class_counts / total_samples
class_weights = 1.0 / class_frequencies

# Normalize the class weights
class_weights /= np.sum(class_weights)

class_weights.sum()

1.0000000000000002

In [25]:
np.unique(df_dataset["tokenClass"], return_counts=True)

(array(['ArithmeticalOperator', 'ArraysClass', 'BOOLEAN',
        'BitwiseLogicalOperator', 'BitwiseNegateOperator',
        'BitwiseShiftOperator', 'Class', 'ClassField', 'ClassModifier',
        'ClosingParenthesis', 'Colon', 'Comma', 'DOUBLE', 'EqOperator',
        'FALSE', 'IneqOperator', 'InstanceOfOperator', 'LogicalOperator',
        'MatchMethod', 'MatchMethodVar', 'MethodArgument', 'MethodName',
        'MethodResultID', 'NULL', 'NonEqIneqOperator',
        'OpeningParenthesis', 'Period', 'QuestionMark', 'RightArrow',
        'S_INT', 'S_STRING', 'Semicolon', 'StreamMethod', 'TRUE', 'This'],
       dtype=object),
 array([  253,  6109,   138,   186,   841,   186, 32428,   239,     7,
         1368,   386,     9,   887,  5149,   828,  4098,  2651,  5729,
           14,    42, 29226,   523,   961,  2358,  1184, 29003,  3778,
          296,    14,  2212,    13, 31038,    52, 12001, 14980]))

In [22]:
len(unique_labels)

35

In [11]:
a = df_dataset[df_dataset["tokenClass"]=="Class"]

In [14]:
a

,label,oracleId,oracleType,projectName,packageName,className,javadocTag,methodJavadoc,methodSourceCode,classJavadoc,classSourceCode,oracleSoFar,token,tokenClass,tokenInfo
143,False,813,EXCEPT_POST,commons-math3-3.6.1,org.apache.commons.math3.linear,MatrixUtils,@throws org.apache.commons.math3.linear.NonSym...,/** * Checks whether a matrix is symmetr...,public static void checkSymmetric(RealMatrix m...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,(,RealFieldElement,Class,"['org.apache.commons.math3', 'RealFieldElement']"
144,False,813,EXCEPT_POST,commons-math3-3.6.1,org.apache.commons.math3.linear,MatrixUtils,@throws org.apache.commons.math3.linear.NonSym...,/** * Checks whether a matrix is symmetr...,public static void checkSymmetric(RealMatrix m...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,(,Primes,Class,"['org.apache.commons.math3.primes', 'Primes']"
145,False,813,EXCEPT_POST,commons-math3-3.6.1,org.apache.commons.math3.linear,MatrixUtils,@throws org.apache.commons.math3.linear.NonSym...,/** * Checks whether a matrix is symmetr...,public static void checkSymmetric(RealMatrix m...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,(,SmallPrimes,Class,"['org.apache.commons.math3.primes', 'SmallPrim..."
146,False,813,EXCEPT_POST,commons-math3-3.6.1,org.apache.commons.math3.linear,MatrixUtils,@throws org.apache.commons.math3.linear.NonSym...,/** * Checks whether a matrix is symmetr...,public static void checkSymmetric(RealMatrix m...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,(,PollardRho,Class,"['org.apache.commons.math3.primes', 'PollardRho']"
147,False,813,EXCEPT_POST,commons-math3-3.6.1,org.apache.commons.math3.linear,MatrixUtils,@throws org.apache.commons.math3.linear.NonSym...,/** * Checks whether a matrix is symmetr...,public static void checkSymmetric(RealMatrix m...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,(,CurveFitter,Class,"['org.apache.commons.math3.fitting', 'CurveFit..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462,False,22109,NORMAL_POST,commons-collections4-4.1,org.apache.commons.collections4,BagUtils,an empty sorted Bag.,/** * Get an empty <code>SortedBag</code...,public static <E> SortedBag<E> emptySortedBag(...,/** * Provides utility methods and decorators...,/* * Licensed to the Apache Software Foundati...,true?methodResultID.equals(,AbstractUntypedIteratorDecorator,Class,"['org.apache.commons.collections4.iterators', ..."
1463,False,22109,NORMAL_POST,commons-collections4-4.1,org.apache.commons.collections4,BagUtils,an empty sorted Bag.,/** * Get an empty <code>SortedBag</code...,public static <E> SortedBag<E> emptySortedBag(...,/** * Provides utility methods and decorators...,/* * Licensed to the Apache Software Foundati...,true?methodResultID.equals(,AbstractOrderedMapIteratorDecorator,Class,"['org.apache.commons.collections4.iterators', ..."
1464,False,22109,NORMAL_POST,commons-collections4-4.1,org.apache.commons.collections4,BagUtils,an empty sorted Bag.,/** * Get an empty <code>SortedBag</code...,public static <E> SortedBag<E> emptySortedBag(...,/** * Provides utility methods and decorators...,/* * Licensed to the Apache Software Foundati...,true?methodResultID.equals(,FilterListIterator,Class,"['org.apache.commons.collections4.iterators', ..."
1465,False,22109,NORMAL_POST,commons-collections4-4.1,org.apache.commons.collections4,BagUtils,an empty sorted Bag.,/** * Get an empty <code>SortedBag</code...,public static <E> SortedBag<E> emptySortedBag(...,/** * Provides utility methods and decorators...,/* * Licensed to the Apache Software Foundati...,true?methodResultID.equals(,SplitMapUtils,Class,"['org.apache.commons.collections4', 'SplitMapU..."


In [14]:
tokenClassDict = {k: i for i, k in enumerate(df_dataset["tokenClass"].unique())}

In [28]:
tokenClassDict

{'MethodArgument': 0,
 'Class': 1,
 'This': 2,
 'TRUE': 3,
 'Semicolon': 4,
 'OpeningParenthesis': 5,
 'ArraysClass': 6,
 'IneqOperator': 7,
 'EqOperator': 8,
 'Period': 9,
 'InstanceOfOperator': 10,
 'NULL': 11,
 'LogicalOperator': 12,
 'NonEqIneqOperator': 13,
 'S_INT': 14,
 'BitwiseNegateOperator': 15,
 'QuestionMark': 16,
 'MethodResultID': 17,
 'MethodName': 18,
 'ClosingParenthesis': 19,
 'Colon': 20,
 'FALSE': 21,
 'StreamMethod': 22,
 'BOOLEAN': 23,
 'Comma': 24,
 'ClassField': 25,
 'BitwiseLogicalOperator': 26,
 'ArithmeticalOperator': 27,
 'BitwiseShiftOperator': 28,
 'DOUBLE': 29,
 'MatchMethod': 30,
 'MatchMethodVar': 31,
 'RightArrow': 32,
 'ClassModifier': 33,
 'S_STRING': 34}

In [46]:
df_dataset['tokenClass'] = df_dataset['tokenClass'].apply(lambda x: tokenClassDict[x])

In [47]:
df_dataset.head()

,label,oracleId,oracleType,projectName,packageName,className,javadocTag,methodJavadoc,methodSourceCode,classJavadoc,classSourceCode,oracleSoFar,token,tokenClass,tokenInfo
0,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,0,
1,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,1,
2,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,2,
3,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,3,
4,True,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,4,


In [20]:
# get the list of column names
cols = list(df_dataset)

a = df_dataset.groupby(['oracleId', 'oracleSoFar'])



In [25]:
b = a["tokenClass"].apply(list)

In [31]:
b.head()

oracleId  oracleSoFar
0                        [MethodArgument, Class, TRUE, Semicolon, Openi...
          (                    [MethodArgument, Class, OpeningParenthesis]
          ((                   [MethodArgument, Class, OpeningParenthesis]
          ((start        [IneqOperator, EqOperator, Period, InstanceOfO...
          ((start==                          [MethodArgument, Class, NULL]
Name: tokenClass, dtype: object

In [50]:
cols

['label',
 'oracleId',
 'oracleType',
 'projectName',
 'packageName',
 'className',
 'javadocTag',
 'methodJavadoc',
 'methodSourceCode',
 'classJavadoc',
 'classSourceCode',
 'oracleSoFar',
 'token',
 'tokenClass',
 'tokenInfo']

In [16]:
a = df_dataset.groupby(['oracleId', 'oracleSoFar'])

In [17]:
a.head()

,label,oracleId,oracleType,projectName,packageName,className,javadocTag,methodJavadoc,methodSourceCode,classJavadoc,classSourceCode,oracleSoFar,token,tokenClass,tokenInfo
0,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,MethodArgument,
1,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,Class,
2,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,This,
3,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,TRUE,
4,True,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,Semicolon,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,False,2799,PRE,commons-math3-3.6.1,org.apache.commons.math3.analysis.polynomials,PolynomialsUtils,@param shift Shift value.,/** * Compute the coefficients of the po...,public static double[] shift(final double[] co...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,,,MethodArgument,
482,False,2799,PRE,commons-math3-3.6.1,org.apache.commons.math3.analysis.polynomials,PolynomialsUtils,@param shift Shift value.,/** * Compute the coefficients of the po...,public static double[] shift(final double[] co...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,,,Class,
483,True,2799,PRE,commons-math3-3.6.1,org.apache.commons.math3.analysis.polynomials,PolynomialsUtils,@param shift Shift value.,/** * Compute the coefficients of the po...,public static double[] shift(final double[] co...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,,,Semicolon,
484,False,2799,PRE,commons-math3-3.6.1,org.apache.commons.math3.analysis.polynomials,PolynomialsUtils,@param shift Shift value.,/** * Compute the coefficients of the po...,public static double[] shift(final double[] co...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,,,OpeningParenthesis,


In [18]:
df = pd.DataFrame({
    'A': ['foo', 'foo', 'bar', 'bar', 'foo', 'foo'],
    'B': ['one', 'one', 'one', 'two', 'two', 'one'],
    'C': [1, 2, 3, 4, 5, 6]
})

# group the DataFrame by columns A and B, and aggregate the values in column C into a list
df['D'] = df.groupby(['A', 'B'])['C'].apply(list)

TypeError: incompatible index of inserted column with frame index

In [20]:
import pandas as pd

# create a sample DataFrame
df = pd.DataFrame({
    'A': ['foo', 'foo', 'bar', 'bar', 'foo', 'foo'],
    'B': ['one', 'one', 'one', 'two', 'two', 'one'],
    'C': [1, 2, 3, 4, 5, 6]
})

# group the DataFrame by columns A and B, and get the unique values in column C
df['D'] = df.groupby(['A', 'B'])['C'].unique().apply(list)

# print the result
print(df)

TypeError: incompatible index of inserted column with frame index

In [41]:
import pandas as pd
# Create a sample DataFrame
data = {'A': ['X', 'X', 'Y', 'Y', 'Z'],
        'B': [1, 1, 1, 2, 1],
        'C': [10, 20, 30, 40, 50]}
df = pd.DataFrame(data)

# Create column D with lists of different values from column C
unique_C = df.groupby(['A', 'B'])['C'].unique()
pd.merge(df, unique_C, on=["A","B"])

In [43]:
pd.merge(df, unique_C, on=["A","B"])

,A,B,C_x,C_y
0,X,1,10,"[10, 20]"
1,X,1,20,"[10, 20]"
2,Y,1,30,[30]
3,Y,2,40,[40]
4,Z,1,50,[50]


In [9]:
df.head()

,A,B,C,D
0,X,1,10,"((X, 1), [10])"
1,X,2,20,"((X, 2), [20])"
2,Y,1,30,"((Y, 1), [30])"
3,Y,2,40,"((Y, 2), [40])"
4,Z,1,50,"((Z, 1), [50])"


In [14]:
df = df.astype({
    'D': 'string'
})

df

In [62]:
import pandas as pd
# Create a sample DataFrame
data = {'A': ['X', 'X', 'Y', 'Y', 'Z'],
        'B': [1, 2, 1, 2, 1],
        'C': [10, 20, 30, 40, 50]}
df = pd.DataFrame(data)

# Create column D with lists of different values from column C
df_dataset['A'] = df_dataset.groupby(['oracleId', 'oracleSoFar'])["tokenClass"]
#df["D"] = df["D"].apply(lambda x: "[" + ", ".join(str(x) for x in list(x)) + "]")
df["D"] = df["D"].apply(lambda x: list(x))

ValueError: Length of values (44589) does not match length of index (189187)

In [54]:
import pandas as pd

# Create column D with lists of different values from column C
unique = df_dataset.groupby(['oracleId', 'oracleSoFar'])['tokenClass'].unique().to_frame()
unique = unique.rename(columns={'tokenClass': 'eligibleTokenClasses'})
pd.merge(df_dataset, unique, on=['oracleId', 'oracleSoFar'])

,label,oracleId,oracleType,projectName,packageName,className,javadocTag,methodJavadoc,methodSourceCode,classJavadoc,classSourceCode,oracleSoFar,token,tokenClass,tokenInfo,eligibleTokenClasses
0,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,MethodArgument,,"[MethodArgument, Class, This, TRUE, Semicolon,..."
1,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,Class,,"[MethodArgument, Class, This, TRUE, Semicolon,..."
2,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,This,,"[MethodArgument, Class, This, TRUE, Semicolon,..."
3,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,TRUE,,"[MethodArgument, Class, This, TRUE, Semicolon,..."
4,True,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,Semicolon,,"[MethodArgument, Class, This, TRUE, Semicolon,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189182,False,2799,PRE,commons-math3-3.6.1,org.apache.commons.math3.analysis.polynomials,PolynomialsUtils,@param shift Shift value.,/** * Compute the coefficients of the po...,public static double[] shift(final double[] co...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,,,MethodArgument,,"[MethodArgument, Class, Semicolon, OpeningPare..."
189183,False,2799,PRE,commons-math3-3.6.1,org.apache.commons.math3.analysis.polynomials,PolynomialsUtils,@param shift Shift value.,/** * Compute the coefficients of the po...,public static double[] shift(final double[] co...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,,,Class,,"[MethodArgument, Class, Semicolon, OpeningPare..."
189184,True,2799,PRE,commons-math3-3.6.1,org.apache.commons.math3.analysis.polynomials,PolynomialsUtils,@param shift Shift value.,/** * Compute the coefficients of the po...,public static double[] shift(final double[] co...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,,,Semicolon,,"[MethodArgument, Class, Semicolon, OpeningPare..."
189185,False,2799,PRE,commons-math3-3.6.1,org.apache.commons.math3.analysis.polynomials,PolynomialsUtils,@param shift Shift value.,/** * Compute the coefficients of the po...,public static double[] shift(final double[] co...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,,,OpeningParenthesis,,"[MethodArgument, Class, Semicolon, OpeningPare..."


In [48]:
unique.head()

oracleId  oracleSoFar
0                        [MethodArgument, Class, TRUE, Semicolon, Openi...
          (                    [MethodArgument, Class, OpeningParenthesis]
          ((                   [MethodArgument, Class, OpeningParenthesis]
          ((start        [IneqOperator, EqOperator, Period, InstanceOfO...
          ((start==                          [MethodArgument, Class, NULL]
Name: tokenClass, dtype: object

In [55]:
unique

eligibleTokenClasses
oracleId oracleSoFar                                                   
0                     [MethodArgument, Class, TRUE, Semicolon, Openi...
         (                  [MethodArgument, Class, OpeningParenthesis]
         ((                 [MethodArgument, Class, OpeningParenthesis]
         ((start      [IneqOperator, EqOperator, Period, InstanceOfO...
         ((start==                        [MethodArgument, Class, NULL]
...                                                                 ...
26710                 [MethodArgument, Class, This, Semicolon, Openi...
26711                 [MethodArgument, Class, This, TRUE, Semicolon,...
26712                 [MethodArgument, Class, This, Semicolon, Openi...
26713                 [MethodArgument, Class, This, TRUE, Semicolon,...
26714                 [MethodArgument, Class, This, TRUE, Semicolon,...

[44589 rows x 1 columns]

In [5]:
import pandas as pd

# Create column D with lists of different values from column C
unique = df_dataset.groupby(['oracleId', 'oracleSoFar'])['tokenClass'].unique().to_frame()
unique = unique.rename(columns={'tokenClass': 'eligibleTokenClasses'})
df_dataset = pd.merge(df_dataset, unique, on=['oracleId', 'oracleSoFar'])



In [64]:
df_dataset.head()

,label,oracleId,oracleType,projectName,packageName,className,javadocTag,methodJavadoc,methodSourceCode,classJavadoc,classSourceCode,oracleSoFar,token,tokenClass,tokenInfo,eligibleTokenClasses_x,eligibleTokenClasses_y,eligibleTokenClasses
0,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,MethodArgument,,"[MethodArgument, Class, This, TRUE, Semicolon,...","[MethodArgument, Class, This, TRUE, Semicolon,...","[MethodArgument, Class, This, TRUE, Semicolon,..."
1,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,Class,,"[MethodArgument, Class, This, TRUE, Semicolon,...","[MethodArgument, Class, This, TRUE, Semicolon,...","[MethodArgument, Class, This, TRUE, Semicolon,..."
2,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,This,,"[MethodArgument, Class, This, TRUE, Semicolon,...","[MethodArgument, Class, This, TRUE, Semicolon,...","[MethodArgument, Class, This, TRUE, Semicolon,..."
3,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,TRUE,,"[MethodArgument, Class, This, TRUE, Semicolon,...","[MethodArgument, Class, This, TRUE, Semicolon,...","[MethodArgument, Class, This, TRUE, Semicolon,..."
4,True,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,,Semicolon,,"[MethodArgument, Class, This, TRUE, Semicolon,...","[MethodArgument, Class, This, TRUE, Semicolon,...","[MethodArgument, Class, This, TRUE, Semicolon,..."


In [6]:
cols = list(df_dataset.columns)

# move column D after column B
cols.insert(cols.index('oracleSoFar') + 1, cols.pop(cols.index('eligibleTokenClasses')))

df_dataset = df_dataset[cols]

In [63]:
cols

['label',
 'oracleId',
 'oracleType',
 'projectName',
 'packageName',
 'className',
 'javadocTag',
 'methodJavadoc',
 'methodSourceCode',
 'classJavadoc',
 'classSourceCode',
 'oracleSoFar',
 'eligibleTokenClasses',
 'token',
 'tokenClass',
 'tokenInfo',
 'eligibleTokenClasses_x',
 'eligibleTokenClasses_y']

In [7]:
df_dataset.head()

,label,oracleId,oracleType,projectName,packageName,className,javadocTag,methodJavadoc,methodSourceCode,classJavadoc,classSourceCode,oracleSoFar,eligibleTokenClasses,token,tokenClass,tokenInfo
0,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,"[MethodArgument, Class, This, TRUE, Semicolon,...",,MethodArgument,
1,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,"[MethodArgument, Class, This, TRUE, Semicolon,...",,Class,
2,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,"[MethodArgument, Class, This, TRUE, Semicolon,...",,This,
3,False,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,"[MethodArgument, Class, This, TRUE, Semicolon,...",,TRUE,
4,True,10700,NORMAL_POST,commons-math3-3.6.1,org.apache.commons.math3.optimization.linear,SimplexTableau,number of constraint with the specified relat...,/** * Get a count of constraints corresp...,private int getConstraintTypeCounts(final Rela...,/** * A tableau for use in the Simplex method...,/* * Licensed to the Apache Software Foundati...,,"[MethodArgument, Class, This, TRUE, Semicolon,...",,Semicolon,


In [16]:
"[" + ", ".join(df_dataset["eligibleTokenClasses"][0]) + "]"

'[MethodArgument, Class, This, TRUE, Semicolon, OpeningParenthesis]'

In [15]:
tokenClassDict = {k: i for i, k in enumerate(df_dataset["tokenClass"].unique())}


In [20]:
value_counts = df_dataset['tokenClass'].value_counts()

# Determine the threshold value based on 5% of the total number of rows
threshold = len(df_dataset) * 0.005

# Filter the DataFrame to keep only the rows meeting the threshold
filtered_df = df_dataset[df_dataset['tokenClass'].map(value_counts) >= threshold]

In [21]:
filtered_df["tokenClass"].unique()

<StringArray>
[    'MethodArgument',              'Class',               'This',
               'TRUE',          'Semicolon', 'OpeningParenthesis',
        'ArraysClass',       'IneqOperator',         'EqOperator',
             'Period', 'InstanceOfOperator',               'NULL',
    'LogicalOperator',  'NonEqIneqOperator',              'S_INT',
     'MethodResultID', 'ClosingParenthesis']
Length: 17, dtype: string

In [23]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')

text = "def greet(user): print(f'hello <extra_id_0>!')"
input_ids = tokenizer(text, return_tensors="pt").input_ids

# simply generate a single sequence
generated_ids = model.generate(input_ids, max_length=8)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))
# this prints "{user.username}"


ConnectionError: HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Read timed out.